In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from transformers import BertTokenizer, BertModel
import torch
from torch.utils.data import DataLoader, TensorDataset

/Users/newswav/Documents/FYPFakeNews/fyp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("fake_or_real_news.csv")
data 

,id,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [3]:
data["fake"] = data['label'].apply(lambda x: 0 if x == "REAL" else 1)

In [4]:
X, y = data["title"], data["fake"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
# vectorizer = TfidfVectorizer(stop_words="english", max_df=0.7)
# X_train_vectorized = vectorizer.fit_transform(X_train)
# X_test_vectorized = vectorizer.transform(X_test)

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset

# Load the DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


Error while downloading from https://cdn-lfs.huggingface.co/distilbert-base-uncased/5e3f1108e3cb34ee048634875d8482665b65ac713291a7e32396fb18f6ff0063?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1710496722&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMDQ5NjcyMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kaXN0aWxiZXJ0LWJhc2UtdW5jYXNlZC81ZTNmMTEwOGUzY2IzNGVlMDQ4NjM0ODc1ZDg0ODI2NjViNjVhYzcxMzI5MWE3ZTMyMzk2ZmIxOGY2ZmYwMDYzP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=NrhfQ1P6CRXyKKt371BcQfp40HN-rwkYmIgONJJEQ2-yeT2pywRDg92oyJ2WCUclSCMZF5tj9je5UM1wTzoAD8UdDPLslCPR3yet9d%7EAD02R2oCZif0wtzidUsFHWQX056qkpOM8YrK%7Emirwje%7Esh89l2xPcjrbetBZZD5TreH6W2TgnS4zz9g1ZjGYJaM0syFpgAD6UKxnw8CMG-pr3SzqncvinqzeUgemHa-DadYw%7EcWlh-j9rrIznsXTbjcqRr%7EooC8ke3zNH%7EXzy0AkBofQPVmpbmJLi3SGA-ofTXxrkXChhLQcsREF7s-HrZDoUex6cPMC%7ETbgmtrc0yxQyRQ__&Key-Pair-I

In [7]:
# clf = LinearSVC()
# clf.fit(X_train_vectorized, y_train)

# knn = KNeighborsClassifier()
# knn.fit(X_train_vectorized, y_train)


# Use the tokenizer in the same way as before
def encode_texts(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

# Assuming data["title"] and y are prepared
X_encoded = encode_texts(data["title"].tolist())

# Prepare the dataset and DataLoader
dataset = TensorDataset(X_encoded['input_ids'], X_encoded['attention_mask'], torch.tensor(y, dtype=torch.long))
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
model.train()
for epoch in range(3):
    for batch in loader:
        batch_input_ids, batch_attention_mask, batch_labels = batch
        model.zero_grad()
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch}, Loss: {loss.item()}")

/Users/newswav/Documents/FYPFakeNews/fyp/lib/python3.12/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0, Loss: 0.7063254714012146
Epoch 0, Loss: 0.703127920627594
Epoch 0, Loss: 0.6827647089958191
Epoch 0, Loss: 0.6642335653305054
Epoch 0, Loss: 0.6687981486320496
Epoch 0, Loss: 0.6992157697677612
Epoch 0, Loss: 0.7021371722221375
Epoch 0, Loss: 0.6436401605606079
Epoch 0, Loss: 0.6330496072769165
Epoch 0, Loss: 0.6883372664451599
Epoch 0, Loss: 0.6454008221626282
Epoch 0, Loss: 0.6444891095161438
Epoch 0, Loss: 0.674267053604126
Epoch 0, Loss: 0.6550200581550598
Epoch 0, Loss: 0.6757447123527527
Epoch 0, Loss: 0.6410918831825256
Epoch 0, Loss: 0.6632018685340881
Epoch 0, Loss: 0.5786011815071106
Epoch 0, Loss: 0.6669766306877136
Epoch 0, Loss: 0.547832190990448
Epoch 0, Loss: 0.626020073890686
Epoch 0, Loss: 0.49234673380851746
Epoch 0, Loss: 0.5363789200782776
Epoch 0, Loss: 0.6965688467025757
Epoch 0, Loss: 0.5639642477035522
Epoch 0, Loss: 0.5298178791999817
Epoch 0, Loss: 0.5883207321166992
Epoch 0, Loss: 0.568130373954773
Epoch 0, Loss: 0.5052759647369385
Epoch 0, Loss: 0.5

In [9]:
model.save_pretrained('./my_distilbert_model')
tokenizer.save_pretrained('./my_distilbert_model')

('./my_distilbert_model/tokenizer_config.json',
 './my_distilbert_model/special_tokens_map.json',
 './my_distilbert_model/vocab.txt',
 './my_distilbert_model/added_tokens.json')

In [10]:

with open("mytext.txt", "r", encoding="utf-8") as f:
    text = f.read()

def get_embeddings(texts):
    # Tokenize texts and prepare input tensors
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # Generate embeddings with DistilBERT
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the mean pooling strategy to get a single vector for each input
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Assuming 'texts' is a list of strings from your dataset
embeddings = get_embeddings([text])

embeddings_np = embeddings.numpy() if isinstance(embeddings, torch.Tensor) else embeddings



AttributeError: 'SequenceClassifierOutput' object has no attribute 'last_hidden_state'